In [ ]:
#importing packages
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn import preprocessing
from sklearn import utils
import collections
import os
import json
import re
import torch
!pip install transformers
from transformers import pipeline

In [ ]:
#TESTING MODEL ACCURACY
misogyny = pipeline("text-classification", model="MilaNLProc/bert-base-uncased-ear-misogyny")
sexism = pipeline("text-classification", model='NLP-LTU/bertweet-large-sexism-detector')
texts = ["I love men", 'What do you think about the gender pay gap, valid right?', 'Girls are weird', 'I\'m having sex with a girl', 'Im having sex']
misogynyResults = []
sexismResults = []
for i in texts:
  misogynyResults.append(misogyny(i))
  sexismResults.append(sexism(i))

#get star from sentAnal
print('Misogyny Results: ')
for i in range(len(misogynyResults)):
  print(misogynyResults[i][0]['label'], ': ', texts[i])

print()
print('Sexism Results: ')
for i in range(len(sexismResults)):
  print(sexismResults[i][0]['label'], ': ', texts[i])

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Misogyny Results: 
non-misogynist :  i love men
misogynist :  what do you think about the gender pay gap, valid right?
misogynist :  girls are weird
misogynist :  im having sex with a girl
non-misogynist :  im having sex

Sexism Results: 
not sexist :  i love men
not sexist :  what do you think about the gender pay gap, valid right?
sexist :  girls are weird
not sexist :  im having sex with a girl
not sexist :  im having sex


In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')
path="/content/drive/My Drive/"
os.chdir(path)
os.listdir(path)
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/data.csv

#1,038,319 posts
#columns: create_time, from_id, from_name, message, post_name
fbdata = pd.read_csv('/content/drive/My Drive/Gender_Studies_Data/fb_news_comments_1000K_hashed.csv')
fbdata = pd.DataFrame(fbdata)

#948,390 comments
#columns: archived, author, author_fullname, body, comment_type,
#         controversiality, created_utc, edited, gilded, id, link_id,
#         locked, name, parent_id, permalink, retrieved_on, score,
#         subreddit_id, subreddit_name_prefixed, subreddit_type,
#         total_awards_received
redditdata = pd.read_parquet('/content/drive/My Drive/Gender_Studies_Data/UpliftingNews-00000-of-00005-52c86ef64bf6875d.parquet')
redditdata = pd.DataFrame(redditdata)

In [ ]:
#select 2017 and later messages from redditdata
redditdatasorted = pd.DataFrame(redditdata.sort_values('created_utc', axis=0))
redditdata2017 = redditdatasorted.iloc[:300000,:].iloc[:100000,:]
redditdata2017 = redditdata2017.sample(frac=1)

fbdata2017 = fbdata.sample(frac=1)

In [ ]:
#evaluate reddit deleted and removed comments
deletedcount = 0
removedcount = 0
for i in range(redditdata2017.shape[0]):
  reddittext = redditdata2017['body'].iloc[i]
  #deal with nulls
  if reddittext != reddittext:
    continue
  if reddittext == '[deleted]':
    deletedcount = deletedcount + 1
  if reddittext == '[removed]':
    removedcount = removedcount + 1

print('percentage of deleted comments: ', deletedcount/redditdata2017.shape[0])
print('percentage of removed comments: ', removedcount/redditdata2017.shape[0])

percentage of deleted comments:  0.09296
percentage of removed comments:  0.10731


In [ ]:
#evaluate misogyny and sexism
limit = 5000
fbmisogynist = 0
redditmisogynist = 0
fbsexist = 0
redditsexist = 0

fbcount = 0
fbindex = 0
redditcount = 0
redditindex = 0

#evaluate facebook comments
while fbcount < limit:
  fbtext = fbdata2017['message'].iloc[fbindex]
  fbindex = fbindex+1
  #remove nulls and too-long messages
  if fbtext != fbtext or len(fbtext) > 511:
    continue
  if misogyny(fbtext)[0]['label'] == 'misogynist':
    fbmisogynist = fbmisogynist+1
  if sexism(fbtext)[0]['label'] == 'sexist':
    fbsexist = fbsexist+1
  fbcount = fbcount + 1

print('number of misogynistic facebook comments: ', fbmisogynist)
print('number of sexist facebook comments: ', fbsexist)

while redditcount < limit:
  reddittext = redditdata2017['body'].iloc[redditindex]
  redditindex = redditindex+1
  #removed nulls, deleted or removed comments, and too-long messages
  if reddittext != reddittext or reddittext == '[deleted]' or reddittext == '[removed]' or len(reddittext) > 511:
    continue
  if misogyny(reddittext)[0]['label'] == 'misogynist':
    redditmisogynist = redditmisogynist+1
  if sexism(reddittext)[0]['label'] == 'sexist':
    redditsexist = redditsexist+1
  redditcount = redditcount+1

print('number of misogynistic reddit comments: ', redditmisogynist)
print('number of sexist facebook comments: ', redditsexist)

350
46
233
41
